In [3]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from random import randint
from collections import OrderedDict 
import numpy as np
import pandas as pd
import random
import sys
import io
import re
from unidecode import unidecode

In [4]:
songs = pd.read_csv('data/nicki-minaj-lyrics.csv')

In [5]:
def get_tokenized_lines(df):
    words = []
    
    for index, row in df['lyrics'].iteritems():
        row = str(row).lower()
        for line in row.split('|-|'):
            new_words = re.findall(r"\b[a-z']+\b", unidecode(line))
            words = words + new_words
        
    return words

In [6]:
all_lyric_lines = get_tokenized_lines(songs)

In [7]:
SEQ_LENGTH = 50 + 1
sequences = list()

for i in range(SEQ_LENGTH, len(all_lyric_lines)):
    seq = all_lyric_lines[i - SEQ_LENGTH: i]
    sequences.append(seq)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 37574


In [8]:
def save_doc(lines, filename):
    for line in lines:
        data = ' '.join(line)
        
    '\n'.join(data)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [9]:
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

In [10]:
vocab = set(all_lyric_lines)
vocab = sorted(vocab)

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}
word_indices = [word_to_index[word] for word in vocab]
vocab_size = len(vocab)

print(word_to_index)
print('vocabulary size: {}'.format(vocab_size))

{"'s": 0, 'a': 1, "a'int": 2, 'aaaand': 3, 'aah': 4, 'aaron': 5, 'abandoning': 6, 'abdomen': 7, 'abort': 8, 'about': 9, 'above': 10, 'abstaining': 11, 'abstract': 12, 'abusive': 13, 'ac': 14, 'account': 15, 'ace': 16, 'acres': 17, 'acrylic': 18, 'act': 19, 'actin': 20, 'acting': 21, 'actually': 22, 'adam': 23, 'add': 24, 'addicted': 25, 'addition': 26, 'addressed': 27, 'administer': 28, 'admit': 29, 'advance': 30, 'afford': 31, 'affordable': 32, 'afraid': 33, 'after': 34, 'again': 35, 'against': 36, 'agendas': 37, 'ago': 38, 'agree': 39, 'ah': 40, 'ahead': 41, 'ahh': 42, 'ahoy': 43, 'aim': 44, 'aiming': 45, "ain't": 46, 'aint': 47, 'air': 48, 'akon': 49, 'aladdin': 50, 'alarm': 51, 'album': 52, 'albums': 53, "alc'hol": 54, 'alcohol': 55, 'alexander': 56, 'ali': 57, 'alice': 58, 'alike': 59, 'alive': 60, 'all': 61, 'alone': 62, 'along': 63, 'alopecia': 64, 'already': 65, 'alright': 66, 'also': 67, 'altar': 68, 'always': 69, 'am': 70, 'ambition': 71, 'american': 72, 'amigos': 73, 'amos':

In [11]:
def get_tokenized_lines(lines, seq_len):
    tokenized = np.zeros((len(lines), seq_len))
    
    for r, line in enumerate(lines):
        for c, word in enumerate(line):
            tokenized[r, c] = word_to_index[word]

    return tokenized

In [12]:
tokenized_seq = get_tokenized_lines(sequences, SEQ_LENGTH)

In [13]:
tokenized_seq[:, -1].shape

(37574,)

In [14]:
from keras.utils import to_categorical

X, y = tokenized_seq[:, :-1], tokenized_seq[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = len(X[0])

In [15]:
print("X_shape", X.shape)
print("y_shape", y.shape)

X_shape (37574, 50)
y_shape (37574, 3894)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, batch_size=128, epochs=200)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            194700    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3894)              393294    
Total params: 738,894
Trainable params: 738,894
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
294/294 [==============================] - 27s 91ms/step - loss: 6.4947 - accuracy: 0.0472
Epoch 

294/294 [==============================] - 31s 106ms/step - loss: 1.6041 - accuracy: 0.6276
Epoch 70/200
294/294 [==============================] - 31s 105ms/step - loss: 1.5849 - accuracy: 0.6322
Epoch 71/200
294/294 [==============================] - 31s 106ms/step - loss: 1.5597 - accuracy: 0.6370
Epoch 72/200
294/294 [==============================] - 31s 105ms/step - loss: 1.5369 - accuracy: 0.6432
Epoch 73/200
294/294 [==============================] - 31s 107ms/step - loss: 1.5284 - accuracy: 0.6437
Epoch 74/200
294/294 [==============================] - 31s 106ms/step - loss: 1.4962 - accuracy: 0.6506
Epoch 75/200
294/294 [==============================] - 31s 105ms/step - loss: 1.4703 - accuracy: 0.6559
Epoch 76/200
294/294 [==============================] - 31s 106ms/step - loss: 1.4494 - accuracy: 0.6618
Epoch 77/200
294/294 [==============================] - 31s 106ms/step - loss: 1.4297 - accuracy: 0.6650
Epoch 78/200
294/294 [==============================] - 32s 107ms/st

294/294 [==============================] - 31s 106ms/step - loss: 0.4762 - accuracy: 0.8825
Epoch 147/200
294/294 [==============================] - 31s 105ms/step - loss: 0.4625 - accuracy: 0.8877
Epoch 148/200
294/294 [==============================] - 31s 107ms/step - loss: 0.4582 - accuracy: 0.8870
Epoch 149/200
294/294 [==============================] - 31s 106ms/step - loss: 0.4368 - accuracy: 0.8918
Epoch 150/200
294/294 [==============================] - 31s 106ms/step - loss: 0.4303 - accuracy: 0.8946
Epoch 151/200
294/294 [==============================] - 31s 105ms/step - loss: 0.4117 - accuracy: 0.9004
Epoch 152/200
294/294 [==============================] - 31s 106ms/step - loss: 0.4051 - accuracy: 0.9032
Epoch 153/200
294/294 [==============================] - 31s 107ms/step - loss: 0.4364 - accuracy: 0.8896
Epoch 154/200
294/294 [==============================] - 31s 105ms/step - loss: 0.4079 - accuracy: 0.8993
Epoch 155/200
294/294 [==============================] - 31s

In [ ]:
model.save('word_model_nicki_minaj.h5')

In [56]:
from keras.models import load_model

model = load_model('word_model_eminem.h5')

In [57]:
def texts_to_sequences(texts, word_to_index):
    indices = np.zeros((1, len(texts)), dtype=int)
    
    for i, text in enumerate(texts):
        if text not in word_to_index:
            random = index_to_word[randint(0,vocab_size)]
            indices[:, i] = word_to_index[random]
        else:
            indices[:, i] = word_to_index[text]
        
    return indices

In [58]:
def my_pad_sequences(seq, maxlen):
    start = seq.shape[1] - maxlen
    
    return seq[:, start: start + maxlen]

In [59]:
def generate_seq(model, word_to_index, seq_length, n_words):
    generated = ''
    usr_input = input("Write the beginning of your poem, the Drake machine will complete it. Your input is: ")
    generated += usr_input 
    sys.stdout.write("\n\nHere is your poem: \n\n") 
    
    result = list()
    in_text = [None] * 51
    generated_list = generated.split()
    
    # if input is shorter than 51 words, fill the beginning with random words
    if(len(generated_list) < 51):
        end = len(generated_list)
        for i in range (51 - end):
            random = index_to_word[randint(0,vocab_size)]
            in_text[i] = random
            
        index = 0
        for i in range (51 - end, 51):
            in_text[i] = generated_list[index]
            index += 1

    # if input is longer than 51 words, only use the last 51 words
    if(len(generated_list) > 51):
        end = len(generated_list)
        in_text = generated_list[end-51:]

    # generate words based on last 50 words
    for _ in range(n_words):
        encoded = texts_to_sequences(in_text[1:], word_to_index)
        encoded = my_pad_sequences(encoded, maxlen=seq_length)
        
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
    
        for word, index in word_to_index.items():
            if index == yhat:
                out_word = word
                break
        
        in_text += ' ' + out_word
        result.append(out_word)
        
    return ' '.join(result)

In [60]:
generated = generate_seq(model, word_to_index, seq_length, 50)
print(generated)

Write the beginning of your poem, the Drake machine will complete it. Your input is: sdahfjashdfkjashd asdfh


Here is your poem: 

['coke', 'cookin', "problem's", 'headache', 'fifth', 'ovulating', 'birth', 'dear', 'neither', 'grips', 'aight', 'pacquiao', 'truthful', 'mann', 'babe', 'misconducts', 'extraterrestrial', 'dodgers', 'smack', 'ruin', 'jade', 'chasing', 'cars', 'poppins', 'tear', 'legendary', 'sorry', 'plenty', 'lewinsky', 'outdo', 'raps', 'growing', 'wide', 'sister', "you'ont", 'opposites', 'winks', 'glide', 'huge', 'southpaw', 'grasp', 'symbolic', 'harris', 'come', 'helped', 'muster', 'sit', 'beer', 'sweden', 'sdahfjashdfkjashd', 'asdfh']
envision fellatio fellow fellatio fellow denver clothes flippity christmas flippity cat triangle uh sa minimum fellatio fellow fellatio fellow fellatio groups resentment lazy hangs pectations jump pigeonholed ho's pinkett clothes pinkett clothes pinkett clothes pinkett lazy hangs bury idaho west pigeonholed fellatio fellow fellatio fellow 